In [22]:
import os

import cv2
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MultiLabelBinarizer

In [51]:
ingredients_path = "../data/nutrition5k_dataset_nosides/metadata/ingredients_metadata.csv"
dish_metadata_path = "test.csv" # "../data/nutrition5k_dataset_nosides/metadata/dish_metadata_cafe1.csv"
dish_ids_path = "../data/nutrition5k_dataset_nosides/dish_ids/dish_ids_all.txt"
img_dir = "../data/nutrition5k_dataset_nosides/imagery/realsense_overhead/"

In [52]:
df = pd.read_csv(ingredients_path)
labels = df["id"]

label_binarizer = MultiLabelBinarizer()
label_binarizer.fit([labels.to_list()])

num_of_classes = label_binarizer.classes_.shape[0]
print("number of classes:", num_of_classes)
label_binarizer.transform([[2, 3]])[0]

number of classes: 555


array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [53]:
def parse_ingredient_id(name: str) -> int:
    return int(name.split('_')[1])

parse_ingredient_id("ingr_0000000008")

8

In [54]:
def get_ingredient_ids_from_row(row):
    ingredient_ids = []

    # Iterate over each ingredient in the row
    # We start from column 7 (0-indexed), and increment by 7 for each ingredient
    for i in range(6, len(row), 7):
        # Check if ingredient id is not missing
        if pd.notnull(row.iloc[i]):
            # Get the ingredient id and remove any leading zeros
            ingredient_ids.append(parse_ingredient_id(row.iloc[i]))
    
    return ingredient_ids

df = pd.read_csv("test2.csv", header=None)
df.head()
row = df.iloc[1]
print(get_ingredient_ids_from_row(row))
print(len(get_ingredient_ids_from_row(row)))

[508, 122, 26, 524, 94, 23, 189, 54, 29, 328, 291, 520, 161, 462, 525, 312, 513]
17


/var/folders/18/mk89nqqd4t1738cgptywpmw00000gn/T/ipykernel_7622/2861585244.py:14: DtypeWarning: Columns (1,2,3,4,5,8,9,10,11,12,15,16,17,18,19,22,23,24,25,26,29,30,31,32,33,36,37,38,39,40,43,44,45,46,47,50,51,52,53,54,57,58,59,60,61,64,65,66,67,68,71,72,73,74,75,78,79,80,81,82,85,86,87,88,89,92,93,94,95,96,99,100,101,102,103,106,107,108,109,110,113,114,115,116,117,120,121,122,123,124,127,128,129,130,131,134,135,136,137,138,141,142,143,144,145,148,149,150,151,152,155,156,157,158,159,162,163,164,165,166,169,170,171,172,173,176,177,178,179,180,183,184,185,186,187,190,191,192,193,194,197,198,199,200,201,204,205,206,207,208,211,212,213,214,215,218,219,220,221,222,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,3

In [55]:
class IngredientDataset(Dataset):
    def __init__(self, img_dir: str, ingredients_path: str, dish_metadata_path: str, dish_ids_path):
        self.img_dir = img_dir
        self.ingredients_path = ingredients_path
        self.dish_metadata_path = dish_metadata_path
        self.dish_ids_path = dish_ids_path

        self.ingredients_df = pd.read_csv(ingredients_path)
        self.label_binarizer = MultiLabelBinarizer()   
        self.label_binarizer.fit([self.ingredients_df["id"].to_list()])

        self.dish_metadata_df = pd.read_csv(dish_metadata_path)
        self.dish_ids_df = pd.read_csv(dish_ids_path, header=None)

    def __len__(self) -> int:
        return self.dish_ids_df.shape[0]

    def __getitem__(self, index):
        dish_id = self.dish_ids_df.iloc[index, 0]

        label = get_ingredient_ids_from_row(self.dish_metadata_path)[self.dish_metadata_path.iloc[:, 0] == dish_id]
        name = self.df.iat[index, 0]

        img_path = os.path.join(self.img_dir, name)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img_tensor = torch.FloatTensor(img)[None, :, :]

        label_encoded = self.label_binarizer.transform([label])[0]
        label_tensor = torch.FloatTensor(label_encoded)
        
        return img_tensor, label_tensor

    def get_num_of_classes(self) -> int:
        return self.label_binarizer.classes_.shape[0]

In [56]:
dataset = IngredientDataset(img_dir, ingredients_path, dish_metadata_path, dish_ids_path)

In [ ]:
dataset[0]